# Minimal Reproduction of Passing Config To Tools In Langgraph

In [1]:
%pip install langgraph langchain-core typing-extensions langchain-openai python-dotenv



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from typing import Literal, Optional, Callable
from langgraph.graph import END, StateGraph
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.prebuilt import ToolNode
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import tools_condition
from langgraph.graph import MessagesState



In [3]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [4]:
def update_dialog_stack(left: list[str], right: Optional[str]) -> list[str]:
    """Push or pop the state"""
    if right is None:
        return left
    if right == "pop":
        return left[:-1]
    return left + [right]

In [5]:
def create_entry_node(assistant_name: str, new_dialog_state: str) -> Callable:
    def entry_node(state: MessagesState) -> dict:
        tool_call_id = state["messages"][-1].tool_calls[0]["id"]
        return {
            "messages": [
                ToolMessage(
                    content=f"The assistant is now the {assistant_name}. Reflect on the above conversation between the host assistant and the user. "  # noqa
                    f"Look at your provided tools to assist the user. Remember, you are {assistant_name},"
                    "If the user changes their mind or needs help which you can't provide with your tools, call the CompleteOrEscalate function to let the primary host assistant take control."  # noqa
                    "Act as the proxy for the assistant.",
                    tool_call_id=tool_call_id,
                )
            ],
            "dialog_state": new_dialog_state,
        }

    return entry_node


## Main Assistant

In [6]:
class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: MessagesState, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)
            if not result.tool_calls and (not result.content or isinstance(result.content, list) and not result.content[0].get("text")):
                messages = state["messages"] + [("user", "Respond with a real output")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


class CompleteOrEscalate(BaseModel):
    """A tool marks the current task as completed and/or to escalate control of the dialog to the main assistant,
    who can re-route the dialog based on the user's needs"""

    cancel: bool = True
    reason: str

    class Config:
        scheme_extra = {
            "example": {"cancel": True, "reason": "User changed their mind about the current task"},
            "example_1": {"cancel": True, "reason": "I have fully completed the task"},
            "example_2": {"cancel": False, "reason": "I have not fully completed the task"},
        }


# Booking Assistant
class BookingAssistant(BaseModel):
    """The Booking Assistant assists manages bookings"""  # noqa

    request: str = Field(description="Any necessary followup questions the booking assistant should clarify before proceeding.")



def get_assistant_runnable():
    model_name = "gpt-4o"
    llm = ChatOpenAI(model=model_name)

    primary_assistant_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You a primary personal assistant."
                "Delegate the task to the appropriate specialized assistant by invoking the corresponding tool. "
                "You are not able to make these types of changes yourself."
            ),
            ("placeholder", "{messages}"),
        ]
    )
    primary_assistant_tools = []
    assistant_runnable = primary_assistant_prompt | llm.bind_tools([BookingAssistant])
    return assistant_runnable, primary_assistant_tools

def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }

def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks([RunnableLambda(handle_tool_error)], exception_key="error")


def pop_dialog_state(state: MessagesState) -> dict:
    messages = []
    if state["messages"][-1].tool_calls:
        messages.append(
            ToolMessage(
                content="Resuming dialog with the host assistant. Please reflect on the past conversation and assist the user as needed.",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        )
    return {
        "dialog_state": "pop",
        "messages": messages,
    }


## Booking Assistant

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.runnables.config import RunnableConfig


@tool
def query_bookings(config: RunnableConfig) -> str:
    """Query only the current user's bookings"""
    
    configuration = config.get("configurable", {})
    print('Configuration:', configuration)
    
    user_id = configuration.get("user_id", None)
    print("User ID:", user_id)
    
    if not user_id:
        return "Failed to query bookings. No User ID configured. Please report back to the user"
    else: 
        return "You have 5 bookings next week."


def get_schedule_assistant_runnable():
    schedule_assistant_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful assistant that helps users make changes or query to their current or historical bookings."
            ),
            ("placeholder", "{messages}"),
        ]
    )
    model_name = "gpt-4o"
    llm = ChatOpenAI(model=model_name)
    booking_safe_tools = [query_bookings]
    booking_runnable = schedule_assistant_prompt | llm.bind_tools(booking_safe_tools + [CompleteOrEscalate])
    return booking_safe_tools, booking_runnable


In [8]:
def build_graph() -> StateGraph:

    builder = StateGraph(MessagesState)

    builder.set_entry_point("primary_assistant")
    booking_safe_tools, booking_runnable = get_schedule_assistant_runnable()
    builder.add_node(
        "enter_booking_assistant",
        create_entry_node("Booking Assistant", "booking_assistant"),
    )
    builder.add_node("booking_assistant", Assistant(booking_runnable))
    builder.add_edge("enter_booking_assistant", "booking_assistant")
    builder.add_node("booking_safe_tools", create_tool_node_with_fallback(booking_safe_tools))

    def route_booking_assistant(
        state: MessagesState,
    ) -> Literal["booking_safe_tools", "leave_skill", "__end__"]:
        route = tools_condition(state)
        if route == END:
            return END
        tool_calls = state["messages"][-1].tool_calls
        did_cancel = any(tc["name"] == CompleteOrEscalate.__name__ for tc in tool_calls)
        if did_cancel:
            return "leave_skill"
        safe_toolnames = [t.name for t in booking_safe_tools]
        if all(tc["name"] in safe_toolnames for tc in tool_calls):
            return "booking_safe_tools"

    builder.add_edge("booking_safe_tools", "booking_assistant")
    builder.add_conditional_edges("booking_assistant", route_booking_assistant)

    # End Query Work Hours And Task Assistant

    builder.add_node("leave_skill", pop_dialog_state)
    builder.add_edge("leave_skill", "primary_assistant")

    assistant_runnable, primary_assistant_tools = get_assistant_runnable()
    builder.add_node("primary_assistant", Assistant(assistant_runnable))
    builder.add_node("primary_assistant_tools", create_tool_node_with_fallback(primary_assistant_tools))

    def route_primary_assistant(
        state: MessagesState,
    ) -> Literal["primary_assistant_tools", "enter_booking_assistant", "__end__"]:
        route = tools_condition(state)
        if route == END:
            return END
        tool_calls = state["messages"][-1].tool_calls
        if tool_calls:
            if tool_calls[0]["name"] == BookingAssistant.__name__:
                return "enter_booking_assistant"
            return "primary_assistant_tools"
        raise ValueError("Invalid Route")

    builder.add_conditional_edges(
        "primary_assistant",
        route_primary_assistant,
        {
            "enter_booking_assistant": "enter_booking_assistant",
            "primary_assistant_tools": "primary_assistant_tools",
            END: END,
        },
    )
    builder.add_edge("primary_assistant_tools", "primary_assistant")
    return builder

In [9]:
builder = build_graph()

graph = builder.compile()

In [10]:
def _print_event(llm_event: dict, _printed: set, max_length=1500):
    current_state = llm_event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = llm_event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)


In [11]:
_printed = set()
message = "What shifts do I have this week? "
response_messages = []
config = {"configurable": {"user_id": 2222}}

llm_events = graph.stream({"messages": ("user", message)}, config, stream_mode="values")
for llm_event in llm_events:
    _print_event(llm_event, _printed)
    response_messages.append(llm_event)
    if "messages" in llm_event and llm_event["messages"]:
        last_message = llm_event["messages"][-1]


================================ Human Message =================================

What shifts do I have this week? 
================================== Ai Message ==================================
Tool Calls:
  BookingAssistant (call_I7o5ijMXKx8sWk0Xe3Kkdt8c)
 Call ID: call_I7o5ijMXKx8sWk0Xe3Kkdt8c
  Args:
    request: Can you provide the shift schedule for this week?
================================= Tool Message =================================

The assistant is now the Booking Assistant. Reflect on the above conversation between the host assistant and the user. Look at your provided tools to assist the user. Remember, you are Booking Assistant,If the user changes their mind or needs help which you can't provide with your tools, call the CompleteOrEscalate function to let the primary host assistant take control.Act as the proxy for the assistant.
================================== Ai Message ==================================
Tool Calls:
  query_bookings (call_ew3Mx0mY0ebMbf34JPvsbh